Careless responder feature engineering and analysis (inital focus on psych_flex survey only)

In [ ]:
#getting and working with data
import pandas as pd
import numpy as np
import re
import os
from itertools import groupby
import datetime as dt
from numpy import median

#visualizing results
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
#import yellowbrick as yb

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import warnings; warnings.simplefilter('ignore')
np.set_printoptions(suppress=True)

In [ ]:
data_path = 'C:/Users/Schindler/Documents/Schindler_Lab/Data/Clinical projects/TILES/final_data/data_final_combined.pkl'

In [ ]:
#read in csv containing participant info
data = pd.read_pickle(data_path)
data = pd.DataFrame(data = data)
#data_PF.reset_index(inplace=True)

print('Original data shape:\n', data.shape, '\n')
#ensure no replicate ID (212 participants in study)
print('Original data unique IDs:\n', data['ParticipantID'].unique().shape, '\n')
#ensure no replicate ID (212 participants in study)
print('Original data unique IDs:\n', data['MitreID'].unique().shape, '\n')
#how much missing data is there?
print('Original data missing value counts:\n', data.isnull().sum(), '\n')
#what is the data type of each column?
print('Original data data types:\n', data.info(), '\n')

In [ ]:
#explore unique values for each column - look for where they need to be updated/cleaned etc
final_cols = data.columns
for col in final_cols:
    print(col)
    print(data[col].unique())
    print('\n')

In [ ]:
#remove pilot participants ('0adb7679-9d26-46e7-a134-11da293910f3', '803be457-cee4-4f3f-9540-ff574c57c697',
# '9519432e-bf05-4f6c-80d7-f61e930f14ba', 'c7acafcf-f5dd-411e-b90c-988279852fb8', 'fae38580-1e2d-4491-9de5-7b75703ba4ce')

data_no_pilot = data[(data['ParticipantID'] != '0adb7679-9d26-46e7-a134-11da293910f3') 
                     & (data['ParticipantID'] != '803be457-cee4-4f3f-9540-ff574c57c697') 
                     & (data['ParticipantID'] != '9519432e-bf05-4f6c-80d7-f61e930f14ba') 
                     & (data['ParticipantID'] != 'c7acafcf-f5dd-411e-b90c-988279852fb8') 
                     & (data['ParticipantID'] != 'fae38580-1e2d-4491-9de5-7b75703ba4ce')]


In [ ]:
#make new column corresponding to the number in series of survey sent for each participant for each survey type
#for psych_flex and engage_psycap this will be count of total surveys sent to each participant
#for MGT surveys this will be a count of the surveys completed for each participant

data_no_pilot = data_no_pilot.sort_values(by=['MitreID', 'survey_type', 'wave_study_day'])

part_id = data_no_pilot['MitreID'].unique()
survey_types = data_no_pilot['survey_type'].unique()

for participant in part_id:
    for survey in survey_types:
        surveys = data_no_pilot[(data_no_pilot['MitreID'] == participant) & (data_no_pilot['survey_type'] == survey)]
        data_no_pilot.loc[(data_no_pilot['MitreID'] == participant) & (data_no_pilot['survey_type'] == survey), 'survey_count'] = np.arange(surveys.shape[0])

In [ ]:
#remove extra surveys
#for psych_flex anything over 49 (eg 50 surveys)
#for engage_psycap anything over 19 (eg 20 surveys)

data = data_no_pilot[((data_no_pilot['survey_type'] == 'engage_psycap') & (data_no_pilot['survey_count'] < 20)) 
                              | ((data_no_pilot['survey_type'] == 'psych_flex') & (data_no_pilot['survey_count'] < 50)) 
                              | ((data_no_pilot['survey_type'] == 'job') & (data_no_pilot['survey_count'] < 30)) 
                              | ((data_no_pilot['survey_type'] == 'health') & (data_no_pilot['survey_count'] < 40)) 
                              | ((data_no_pilot['survey_type'] == 'personality') & (data_no_pilot['survey_count'] < 10))]


In [ ]:
data['survey_type'].value_counts()

In [ ]:
#should be 71
len(data['wave_study_day'].unique())

In [ ]:
#fix numeric data columns
data['activity_num'] = pd.to_numeric(data['activity_num'])
data['location_num'] = pd.to_numeric(data['location_num'])

data['pf_mgt'] = pd.to_numeric(data['pf_mgt'])

data['context1'] = pd.to_numeric(data['context1'])
data['context2'] = pd.to_numeric(data['context2'])
data['context3'] = pd.to_numeric(data['context3'])
data['context4'] = pd.to_numeric(data['context4'])
data['stress'] = pd.to_numeric(data['stress'])
data['anxiety'] = pd.to_numeric(data['anxiety'])
data['work'] = pd.to_numeric(data['work'])

In [ ]:
#create day of week column using dt and wave_study_date
data['day_of_week'] = data['wave_study_date'].dt.day_name()
data['day_of_week'] = pd.Categorical(data['day_of_week'], categories=
    ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday', 'Sunday'],
    ordered=True)

In [ ]:
#create new column corresponding to if they are working or not (as assessed by 'activity_num' == 0 in psych_flex and engage or 
# activity_num == 1 for MGT surveys)

working = []
yes = 1.0
no = 0.0
for index, row in data.iterrows():
    
    if (row['survey_type'] == 'psych_flex') | (row['survey_type'] == 'engage_psycap'):
        
        if row['activity_num'] == 0.0:
            working.append(1.0)
            
        elif row['activity_num'] == -1.0:
            working.append(0.0)
            
        elif row['activity_num'] > 0.0:
            working.append(0.0)
        
        else:
            working.append(np.nan)
            
    elif (row['survey_type'] == 'job') | (row['survey_type'] == 'health') | (row['survey_type'] == 'personality'): 
        
        if row['activity_num'] == 1.0:
            working.append(1.0)
            
        elif row['activity_num'] > 1.0:
            working.append(0.0)
            
        else:
            working.append(np.nan)
    
    else:
        working.append(np.nan)
        
print(len(working))
data['working'] = working

In [ ]:
#create new column corresponding to if they are at work or not (as assessed by 'location_num' == 1 in engage or 
#location_num == 2 in MGT surveys; psych_flex did not ask about location)

at_work = []
yes = 1.0
no = 0.0
for index, row in data.iterrows():
    
    if (row['survey_type'] == 'engage_psycap'):
        
        if row['location_num'] == 1.0:
            at_work.append(1.0)
            
        elif row['location_num'] < 1.0:
            at_work.append(0.0)
            
        elif row['location_num'] > 1.0:
            at_work.append(0.0)
        
        else:
            at_work.append(np.nan)
            
    elif (row['survey_type'] == 'job') | (row['survey_type'] == 'health') | (row['survey_type'] == 'personality'): 
        
        if row['location_num'] == 2.0:
            at_work.append(1.0)
            
        elif row['location_num'] > 2.0:
            at_work.append(0.0)
        elif row['location_num'] < 2.0:
            at_work.append(0.0)
            
        else:
            at_work.append(np.nan)
    
    else:
        at_work.append(np.nan)
        
print(len(at_work))
data['at_work'] = at_work

### notes for CR features for Psych Flex

Should have answered every question

Longstring
- Legitimate longstrings of  ≥ 8 are unlikely for response “5”
    - make column with longest string
    - make column with number that longest string consisted of

Semantic consistency
- Legitimate scores of pf_mgt=5 are almost impossible

Semantic antonyms
- Not applicable

Semantic synonyms 
- not applicable 


In [ ]:
#split off completed PF and related columns
psych_flex_only = data[(data['survey_type'] == 'psych_flex') & (data['completed'] == 1.0)]
psych_flex_only = psych_flex_only[['index', 'MitreID', 'ParticipantID', 'PrimaryUnit', 'SmartPhone',
       'Sex', 'Shift', 'Wave', 'survey_id', 'survey_type', 'date_time',
       'date', 'time', 'Timesent', 'completed', 'start_delay',
       'time_to_complete', 'activity_num', 'location_num',
       'activity', 'pf_03', 'pf_04', 'pf_05', 'pf_06',
       'pf_07', 'pf_08', 'pf_09', 'pf_10', 'pf_11', 'pf_12', 'pf_13',
       'pf_14', 'pf_15', 'pf_mgt', 'exp_0', 'exp_1', 'exp_2', 'exp_3',
       'exp_4', 'exp_5', 'exp_6', 'exp_7', 'exp_8', 'exp_9', 'exp_10',
       'exp_11', 'exp_12', 'exp_13', 'exp_neg', 'exp_pos', 'exp_neut',
       'wave_study_date', 'wave_study_day',
       'survey_count', 'day_of_week', 'working', 'at_work', 'pf_mgt_bin']]

psych_flex_only.shape

In [ ]:
#create feature that is count of missing values from survey

psych_flex_only.loc[psych_flex_only.loc[:, 'activity':'pf_mgt'].isnull().sum(axis=1) > 0, 'missing_values'] = psych_flex_only.loc[:, 'activity':'pf_mgt'].isnull().sum(axis=1)
psych_flex_only.loc[psych_flex_only.loc[:, 'activity':'pf_mgt'].isnull().sum(axis=1) == 0, 'missing_values'] = 0.0

In [ ]:
#create feature that is number of surveys that were not completed

part_id = psych_flex_only['ParticipantID'].unique()

for participant in part_id:
    missing = 49 - psych_flex_only[psych_flex_only['ParticipantID'] == participant]['survey_count'].max()
    psych_flex_only.loc[(psych_flex_only['ParticipantID'] == participant), 'uncompleted'] = missing

In [ ]:
#long string analysis (e.g. max length of same number answered for pf_03:pf_15)
#create features related to long string analysis (feature of how long the string is and feature of what the string consisted of)

max_strings = []
max_nums = []

for index, row in psych_flex_only.iterrows():
    repeats = []
    for k, g in groupby(row.loc['pf_03':'pf_15']):
        repeats.append(sum(1 for i in g))
    max_string = max(repeats)
    
    max_index = repeats.index(max_string)
    max_num = row.loc['pf_03':'pf_15'][max_index]
    
    max_strings.append(max_string)
    
    max_nums.append(max_num)

psych_flex_only['longest_string'] = max_strings
psych_flex_only['longest_string_answer'] = max_nums

In [ ]:
#create CR summary feature (count of positive CR parameters)

counts = []

for index, row in psych_flex_only.iterrows():
    count = 0
    if row['missing_values'] > 0:
        count += 1
    #if row['uncompleted'] > 0:
        #count += 1
    if row['longest_string'] > 7:
        count += 1
    
    counts.append(count)

psych_flex_only['CR_count'] = counts

In [ ]:
#save to csv
psych_flex_only.to_csv('psych_flex_only_CR.csv')

In [ ]:
psych_flex_only['CR_count'].value_counts()

In [ ]:
psych_flex_only.columns.values

In [ ]:
ind_vars = ['time_to_complete', 'pf_mgt', 'missing_values', 'uncompleted',
       'longest_string_answer', 'longest_string', 'CR_count']
psych_flex_only[ind_vars].corr()

In [ ]:
plt.figure(figsize=(25, 10))
sns.catplot(x="longest_string", y="time_to_complete", data=psych_flex_only, kind="bar")

In [ ]:
plt.figure(figsize=(25, 10))
sns.catplot(x="longest_string", y="pf_mgt", data=psych_flex_only, kind="bar")

In [ ]:
plt.figure(figsize=(25, 10))
sns.catplot(x="wave_study_day", y="longest_string", data=psych_flex_only, col='Shift', kind="bar")

In [ ]:
g = sns.FacetGrid(psych_flex_only, col="survey_count", col_wrap=7, height=7)
g.map(sns.barplot, 'ParticipantID', "longest_string")

In [ ]:
plt.figure(figsize=(25, 10))
sns.catplot(x="ParticipantID", y="longest_string", data=psych_flex_only.sort_values('longest_string'), kind="bar")

In [ ]:
plt.figure(figsize=(25, 20))
sns.catplot(x="ParticipantID", y="longest_string", data=psych_flex_only[psych_flex_only['wave_study_day'] < 15].sort_values('longest_string'), kind="bar")

In [ ]:
plt.figure(figsize=(25, 10))
sns.catplot(x="ParticipantID", y="longest_string", data=psych_flex_only[psych_flex_only['survey_count'] < 15].sort_values('ParticipantID'), kind="bar")

In [ ]:
plt.figure(figsize=(25, 10))
sns.catplot(x="ParticipantID", y="longest_string", data=psych_flex_only[psych_flex_only['survey_count'] > 40].sort_values('ParticipantID'), kind="bar")

In [ ]:
plt.figure(figsize=(25, 10))
sns.catplot(x="CR_count", y="time_to_complete", data=psych_flex_only, kind="bar")

### notes for CR features for MGT survey context 1-4 questions

-	context1 = “location”on engage_psycap but context1/location not available for psych_flex
-	context2 = “activity” on psych_flex and “engage_activity” on engage_psycap
-	context3 and context4 not available for psych_flex or engage_psycap
-	please change variable names to be consistent across surveys: activity, location
-	please change response labels to be consistent across surveys: begin with “1” 


Longstring analyses
-	not applicable 


Semantic Antonyms
- if context1 = home (1), then context2 ≠ work and work related (1)
- if context1 = work (2), then context2 ≠ leisure sports (3), household activities (6), org/civic (10)


Semantic Synonyms
- if context1 = work (2), then context2 most likely work and work related (1)
- If context1 = vehicle (5), then context2 most likely travel or commute (11)
